### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
query = 'movie reviews'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}")
print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=a6dDcvdk7yBatG4Pm3r20Kb0gZqnuPSS


In [6]:
#https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=a6dDcvdk7yBatG4Pm3r20Kb0gZqnuPSS&page=0

In [7]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page in range(0, 20):
    #query_url = f"{url}api-key={nyt_api_key}&q={query}&begin_date={begin_date}&end_date={end_date}"
    #query_url = (f'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=a6dDcvdk7yBatG4Pm3r20Kb0gZqnuPSS')
    query_url = (f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}")

    # Create query with a page number
    # API results show 10 articles at a time
    query_url = f"{query_url}&page={str(page)}"
    #print(query_url)
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # Loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Page {page} retrieved successfully.")
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"Page {page} had no results.")
        break

# Print the total number of reviews retrieved
print(f"Total reviews retrieved: {len(reviews_list)}")
# issue is somehow its storing the same first page 20 times in the list, so the same 10 movies are repeating
# fix this

Page 0 retrieved successfully.
Page 1 retrieved successfully.
Page 2 retrieved successfully.
Page 3 retrieved successfully.
Page 4 retrieved successfully.
Page 5 retrieved successfully.
Page 6 retrieved successfully.
Page 7 retrieved successfully.
Page 8 retrieved successfully.
Page 9 retrieved successfully.
Page 10 retrieved successfully.
Page 11 retrieved successfully.
Page 12 retrieved successfully.
Page 13 retrieved successfully.
Page 14 retrieved successfully.
Page 15 retrieved successfully.
Page 16 retrieved successfully.
Page 17 retrieved successfully.
Page 18 retrieved successfully.
Page 19 retrieved successfully.
Total reviews retrieved: 200


In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_results = reviews_list[:5]
formatted_results = json.dumps(preview_results, indent=4)


In [9]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = json_normalize(reviews_list)


In [10]:
''' The problem with the given function give here is that I ended up finding some reviews
that used a different delimiter than those unicode characters, so I made another loop 
to take care of that case. There some more edge cases, but I felt it wasn't worth spending
more time on. TL;DR I didn't use the given lambda because the function I made worked better
for the inconsistencies in the data I encountered.
'''
# lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]

# Function to extract the title
def extract_title(headline):
    if pd.isna(headline):
        return ""
# # Title is between unicode characters \u2018 and \u2019.
#     start = headline.find('\u2018') + 1
#     end = headline.find('\u2019')

#     if start == 0 or end == -1:
#         start = headline.find("'") + 1
#         end = headline.find("'", start)

    
#     if start > 0 and end > start:
# # End string should include " Review" to avoid cutting title early
#         return headline[start:end] #+ " Review"
#     return headline

    # Title is between unicode characters \u2018 and \u2019.
    start = headline.find('\u2018') + 1
    end = headline.find('\u2019')

    if start == 0 or end == -1:
        # Fallback to using regular single quotes if the unicode quotes are not found
        start = headline.find("'") + 1
        end = headline.find("'", start)

    if start > 0 and end > start:
        title = headline[start:end]
    else:
        title = headline
    
    # Additional check for "Review: " or "Movie Review: " prefix
    review_prefixes = ["Review: ", "Movie Review: "]
    for prefix in review_prefixes:
        if title.startswith(prefix):
            title = title[len(prefix):]
            break
    
    return title.strip()
    

# Extract the title from the "headline.main" column and
# save it to a new column "title"
df_reviews['title'] = df_reviews['headline.main'].apply(extract_title)

df_reviews[['headline.main', 'title']].tail()
titles_list = df_reviews['title'].to_list()
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,It’s All Mirth and Taxes in ‘Love & Taxes’
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)

# Display the DataFrame with the fixed "keywords" column
df_reviews[['keywords']].head()


,keywords
0,subject: Movies;creative_works: The Attachment...
1,"subject: Movies;persons: Kapur, Shekhar;person..."
2,subject: Movies;creative_works: You Can Live F...
3,subject: Movies;creative_works: A Tourist's Gu...
4,"subject: Movies;persons: Zlotowski, Rebecca;cr..."


In [12]:
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,It’s All Mirth and Taxes in ‘Love & Taxes’
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [13]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = df_reviews['title'].to_list()
titles_list

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love 

### Access The Movie Database API

In [15]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="
#query = "action"
tmdb_key_string = "&api_key=" + tmdb_api_key
movie_url = tmdb_url  + tmdb_key_string
movie_url

'https://api.themoviedb.org/3/search/movie?query=&api_key=aab8aada07f42dfe8906834df0736c5a'

In [16]:

# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter and request_counter % 50 == 0:
        print("Sleeping for 60 seconds to respect API limits.")
        time.sleep(60)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    movie_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    movies = requests.get(movie_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movies["results"][0]["id"]

        # Make a request for the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        details = requests.get(details_url).json()
        
        # Extract the genre names into a list
        genres = [genre["name"] for genre in details.get("genres", [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language["english_name"] for language in details.get("spoken_languages", [])]

        # convert the list to a string, this became annoying at the end when it was a list
        spoken_languages_str = ', '.join(spoken_languages)
        
        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in details.get("production_countries", [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            "title": details.get("title"),
            "original_title": details.get("original_title"),
            "budget": details.get("budget"),
            "genres": genres,
            "language": details.get("original_language"),
            "spoken_languages": spoken_languages_str,
            "homepage": details.get("homepage"),
            "overview": details.get("overview"),
            "popularity": details.get("popularity"),
            "runtime": details.get("runtime"),
            "revenue": details.get("revenue"),
            "release_date": details.get("release_date"),
            "vote_average": details.get("vote_average"),
            "vote_count": details.get("vote_count"),
            "production_countries": production_countries
        }
        tmdb_movies_list.append(movie_data)
        
        # Print out the title that was found
        print(f"Found {title}")

    except (IndexError, KeyError):
        print(f"{title} not found or data missing.")

# Print the total number of movies retrieved
print(f"Total movies retrieved: {len(tmdb_movies_list)}")


Found The Attachment Diaries
Found What
Found You Can Live Forever
Found A Tourist
Found Other People
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found The In Between
Found Book of Love
Found Lingui, the Sacred Bond

In [17]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_tmdb = tmdb_movies_list[:5]
formatted_tmdb = json.dumps(preview_tmdb, indent=4)

In [18]:
# Convert the results to a DataFrame
tmdb_df = json_normalize(tmdb_movies_list)

tmdb_df.tail()

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
190,Frantz,Frantz,0,"[History, Drama, Romance]",fr,"German, French",,"In the aftermath of WWI, a young German who gr...",24.700,113,7478354,2016-09-07,7.365,671,"[France, Germany]"
191,The Other Half,The Other Half,0,"[Drama, Romance]",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.240,103,0,2016-12-02,6.300,24,[Canada]
192,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"Turkish, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.898,111,413844,2017-03-10,6.164,247,"[Turkey, United States of America]"
193,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"[Romance, Comedy]",en,"English, Spanish",,"On the surface, Clara Barron seems to have it ...",10.475,102,0,2017-02-10,6.538,159,[Mexico]
194,Kedi,Kedi,0,[Documentary],tr,Turkish,http://www.kedifilm.com,"A profile of Istanbul and its unique people, s...",16.075,79,5037923,2017-01-08,7.556,310,[United States of America]


### Merge and Clean the Data for Export

In [20]:
# Merge the New York Times reviews and TMDB DataFrames on title
df = pd.merge(df_reviews, tmdb_df, on='title', how='inner')
df[['runtime', 'revenue', 'release_date', 'vote_average', 'vote_count',
       'production_countries']]

# keywords, byline.person, genres, production_countries

,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,102,0,2021-10-07,3.000,4,[Argentina]
1,96,15055,2023-03-24,6.500,38,"[Canada, United States of America]"
2,97,93000,2016-09-09,6.312,194,[United States of America]
3,100,37820,2023-04-07,6.513,77,"[Czech Republic, United States of America]"
4,95,77145,2023-04-13,6.000,2,[United States of America]
...,...,...,...,...,...,...
140,83,0,2017-01-14,6.022,114,"[Belgium, France]"
141,108,0,2016-10-07,6.100,330,[Australia]
142,133,12448676,2016-12-02,6.900,456,"[Spain, United States of America]"
143,103,0,2016-12-02,6.300,24,[Canada]


In [21]:
df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title', 'original_title', 'budget', 'genres',
       'language', 'spoken_languages', 'homepage', 'overview', 'popularity',
       'runtime', 'revenue', 'release_date', 'vote_average', 'vote_count',
       'production_countries'],
      dtype='object')

In [22]:
df.dtypes

web_url                     object
snippet                     object
source                      object
keywords                    object
pub_date                    object
word_count                   int64
headline.main               object
headline.kicker             object
headline.content_kicker     object
headline.print_headline     object
headline.name               object
headline.seo                object
headline.sub                object
byline.original             object
byline.person               object
byline.organization         object
title                       object
original_title              object
budget                       int64
genres                      object
language                    object
spoken_languages            object
homepage                    object
overview                    object
popularity                 float64
runtime                      int64
revenue                      int64
release_date                object
vote_average        

In [23]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
fix_cols = ['keywords', 'byline.person', 'genres', 'production_countries']

# Create a list of characters to remove
rm_char = ['"','"','[',']','{','}','(',')']

# Loop through the list of columns to fix
for col in fix_cols:
    # Convert the column to type 'str'
    df[col] = df[col].astype(str)
    
    # Loop through characters to remove
    for char in rm_char:
        df[col] = df[col].str.replace(char, '')

# Display the fixed DataFrame
df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,Spanish,,"Argentina, 1970s. A desperate young woman goes...",4.080,102,0,2021-10-07,3.000,4,'Argentina'
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.650,96,15055,2023-03-24,6.500,38,"'Canada', 'United States of America'"
2,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,English,,"David, a struggling comedy writer fresh off fr...",15.960,97,93000,2016-09-09,6.312,194,'United States of America'
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke 1989- ;p...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,"English, Spanish",,Emma and Jesse are living the perfect life tog...,16.082,100,37820,2023-04-07,6.513,77,"'Czech Republic', 'United States of America'"
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,27.253,95,77145,2023-04-13,6.000,2,'United States of America'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris Movie;subject: M...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,French,,Fiona visits Paris for the first time to assis...,9.965,83,0,2017-01-14,6.022,114,"'Belgium', 'France'"
141,https://www.nytimes.com/2017/05/11/movies/houn...,This feature debut of the Australian filmmaker...,The New York Times,subject: Movies;creative_works: Hounds of Love...,2017-05-11T20:16:51+0000,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,...,English,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,12.374,108,0,2016-10-07,6.100,330,'Australia'
142,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise Mo...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,"Armenian, German, English, French",http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,44.073,133,12448676,2016-12-02,6.900,456,"'Spain', 'United States of America'"
143,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creati

In [24]:
# Drop "byline.person" column
df.drop(columns=['byline.person'], inplace=True)
#f.columns

In [26]:
# Delete duplicate rows and reset index

df = df.drop_duplicates()
df = df.reset_index(drop=True)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,Spanish,,"Argentina, 1970s. A desperate young woman goes...",4.080,102,0,2021-10-07,3.000,4,'Argentina'
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.650,96,15055,2023-03-24,6.500,38,"'Canada', 'United States of America'"
2,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,English,,"David, a struggling comedy writer fresh off fr...",15.960,97,93000,2016-09-09,6.312,194,'United States of America'
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke 1989- ;p...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,"English, Spanish",,Emma and Jesse are living the perfect life tog...,16.082,100,37820,2023-04-07,6.513,77,"'Czech Republic', 'United States of America'"
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,27.253,95,77145,2023-04-13,6.000,2,'United States of America'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris Movie;subject: M...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,French,,Fiona visits Paris for the first time to assis...,9.965,83,0,2017-01-14,6.022,114,"'Belgium', 'France'"
133,https://www.nytimes.com/2017/05/11/movies/houn...,This feature debut of the Australian filmmaker...,The New York Times,subject: Movies;creative_works: Hounds of Love...,2017-05-11T20:16:51+0000,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,...,English,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,12.374,108,0,2016-10-07,6.100,330,'Australia'
134,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise Mo...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,"Armenian, German, English, French",http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,44.073,133,12448676,2016-12-02,6.900,456,"'Spain', 'United States of America'"
135,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creati

In [27]:
# Export data to CSV without the index
df.to_csv('output/output.csv', index=False)
